<a href="https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/NNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# initialize & load

In [1]:
from google.colab import drive
from google.colab import files

import numpy as np
import pandas as pd

from keras import models
from keras import layers
from keras.models import load_model

Using TensorFlow backend.


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  output_LGB.csv  user_test.csv    user_train.csv
model_NN.h5    output_NN.csv   user_test_n.csv  user_train_n.csv


In [5]:
%%time
filename = 'model_NN.h5'
model = load_model(filename)

df_train = pd.read_csv('user_train_n.csv', index_col = 0)
df_test = pd.read_csv('user_test_n.csv', index_col = 0)

df_y = df_train['transactionRevenue']
df_x = df_train.drop(['transactionRevenue'], axis=1)
df_tgt = df_test.drop(['transactionRevenue'], axis=1)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 8.7 s, sys: 867 ms, total: 9.57 s
Wall time: 13.4 s


In [6]:
print(df_x.shape, df_y.shape, df_tgt.shape)

(714167, 23) (714167,) (617242, 23)


# train model

In [0]:
from keras import optimizers


def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(120, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizers.adam(lr=0.01), loss='mse', metrics=['mae'])
    return model

In [0]:
%%time
epochs = 200
batch_size = df_x.shape[0] // 5
validation_split = 0.1

model = build_model(df_x.shape[1])

history = model.fit(df_x,
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split = validation_split,
                    verbose=1)

# evaluate model

In [0]:
#history.history

In [7]:
%%time

from sklearn.metrics import mean_squared_error

pred = model.predict(df_x)
mse = mean_squared_error(df_y, pred)
rmse = np.sqrt(mse)
print(rmse)

2.048579998484373
CPU times: user 36.2 s, sys: 3.91 s, total: 40.1 s
Wall time: 32.5 s


2.048579998484373

In [0]:
%%time
model.save(filename)

#files.download(filename)

# predict & output

In [0]:
%%time
pred_tgt = model.predict(df_tgt)


In [0]:
%%time

pred_tgt[pred_tgt<0] = 0
df_out = pd.DataFrame(index = df_tgt.index)
df_out['PredictedLogRevenue'] = pred_tgt

df_out.to_csv('output_NN.csv')

In [0]:
pred_tgt.shape, pred_tgt.max(), pred_tgt.min(), len(pred_tgt[pred_tgt>=1.5])/len(pred_tgt)

In [0]:
print(len(df_y[df_y<=0.1]) - len(df_y[df_y==0]))

print(len(pred_tgt[pred_tgt<=0.1]) - len(pred_tgt[pred_tgt==0]))

print(len(df_y[df_y>0]) / len(df_y) )

print(len(pred_tgt[pred_tgt>1]) / len(pred_tgt) )

In [0]:
df_out.head()

In [0]:
df_out.describe()